In [81]:
import pandas as pd
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [82]:
# load in the data as all strings

In [91]:
df_train = pd.read_csv("../data/emrqa_context_train_predicted.csv", dtype=str)
df_val = pd.read_csv("../data/emrqa_context_val_predicted.csv", dtype=str)
df_test = pd.read_csv("../data/emrqa_context_test_predicted.csv", dtype=str)

In [92]:
def calc_token_spans(question, evidence, answer):
    if not evidence or evidence == "NaN":
        return 0, 0
    inputs = tokenizer(question, evidence, return_offsets_mapping=True)
    offset = inputs['offset_mapping']
    calc_answer_start = evidence.find(answer)
    end_char = calc_answer_start + len(answer)
    sequence_ids = inputs.sequence_ids()

    if calc_answer_start == -1:
        return 0, 0

    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    idx = context_start
    try:
        while idx <= context_end and offset[idx][0] <= calc_answer_start:
            idx += 1
        start_position = idx - 1

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_position = idx + 1
    except:
        print(offset)
        print(idx)
        print(len(offset))
        print(offset[idx])

    # if start_position == end_position:
    #     return 0, 0
    if start_position >= 512 or end_position >= 512:
        return 0, 0
    return start_position, end_position

In [93]:
def convert_to_correct_data(df):
    df = df[['question', 'predicted_evidence', 'answer']] 
    df[['start_token', 'end_token']] = df.apply(lambda x: calc_token_spans(str(x['question']), str(x['predicted_evidence']), str(x['answer'])), axis=1, result_type='expand')
    return df

In [94]:
df_train = convert_to_correct_data(df_train)
df_val = convert_to_correct_data(df_val)
df_test = convert_to_correct_data(df_test)

C:\Users\knigh\AppData\Local\Temp\ipykernel_25096\1094077.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['start_token', 'end_token']] = df.apply(lambda x: calc_token_spans(str(x['question']), str(x['predicted_evidence']), str(x['answer'])), axis=1, result_type='expand')
C:\Users\knigh\AppData\Local\Temp\ipykernel_25096\1094077.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['start_token', 'end_token']] = df.apply(lambda x: calc_token_spans(str(x['question']), str(x['predicted_evidence']), s

In [95]:
df_train

,question,predicted_evidence,answer,start_token,end_token
0,what interventions were done for the patient's...,"postoperatively , she was again noted to have ...",thrombectomy,57,61
1,how often does the patient take econazole nitrate,"daily , hold\nfor systolic less than 90 , hear...",daily,14,14
2,why is the patient taking ocean spray ( sodium...,potentially serious interaction: warfarin &amp...,other:dried nasal mucosa,89,95
3,why is the patient on oxycodone,the patient occasionally takes\noxycodone for ...,fibroid pain.,22,26
4,has the patient ever screened postive on right...,"ekg showed atrial fibrillation , moderate vent...",ekg,18,20
...,...,...,...,...,...
24133,what is the dosage of the medication the patie...,po ( ref #\n73021085 )\npotentially serious in...,lorazepam,51,54
24134,what diagnosis was used for the patient's righ...,she was counseled to follow-up with her pcp f ...,her chest ct,52,55
24135,how often does the patient take micronase,disposition: home\ndischarge medications:\nami...,bid,25,25
24136,what is the dosage of the medication the patie...,"during the\nprevious admission , she had been ...",amiodarone,49,52


In [96]:
# convert nan data in the predicted evidence to empty string
df_train['predicted_evidence'] = df_train['predicted_evidence'].fillna("")
df_val['predicted_evidence'] = df_val['predicted_evidence'].fillna("")
df_test['predicted_evidence'] = df_test['predicted_evidence'].fillna("")

In [97]:
df_train.to_csv("../data/emrqa_train_bert_predicted.csv", index=False)
df_val.to_csv("../data/emrqa_val_bert_predicted.csv", index=False)
df_test.to_csv("../data/emrqa_test_bert_predicted.csv", index=False)

In [89]:
df_train['question'].isna().sum()

0

question              0
predicted_evidence    0
answer                0
start_token           0
end_token             0
dtype: int64